In [1]:
#import statements
#necessary for generator
import csv
import math
import pandas as pd
import random
import copy
#from a Twilio file to pull credentials(previosly must create a file)
# from twilio.rest import TwilioRestClient
# from credentials import account_sid, auth_token, my_cell, my_twilio
#necessary for schedule scraper
import gspread
from datetime import date
import calendar
#from Google import Create_Service


#parsing the csv datafile
def process_csv(filename):
    exampleFile = open(filename, encoding="utf-8")  
    exampleReader = csv.reader(exampleFile) 
    exampleData = list(exampleReader)        
    exampleFile.close()  
    return exampleData
data=process_csv("fit_data.csv")
fit_data=pd.DataFrame(data[18:])


In [2]:
#BOT CREATION STARTS BELOW

In [3]:
#PART ONE, INPUT NECESSARY

#Creating values for each day's workout routine per muscle group 
back=fit_data[(fit_data[1]=="Back - Latissimus Dorsi") | (fit_data[1]=="Back - Lat.Dorsi/Rhomboids")]
back_exercises=list(back[2])

chest=fit_data[(fit_data[1].str.contains("Chest"))]
chest_exercises=list(chest[2])


bis=fit_data[(fit_data[1]=="Biceps")]
bicep_exercises=list(bis[2])

abdominal=fit_data[(fit_data[1].str.contains("Abdominals"))]
abdominal_exercises=list(abdominal[2])

quads=fit_data[(fit_data[1].str.contains("Quadriceps"))]
quad_exercises=list(quads[2])

hams=fit_data[(fit_data[1].str.contains("Hamstrings"))]
hams_exercises=list(hams[2])

triceps=fit_data[(fit_data[1]=="Triceps")]
tricep_exercises=list(triceps[2])

shoulders=fit_data[fit_data[1].str.contains("Shoulder")]
shoulder_exercises=list(shoulders[2])

posterior=fit_data[fit_data[1].str.contains("Erector Spinae")]
lower_back_exercises=list(posterior[2])

calves=fit_data[fit_data[1].str.contains("Calves")]
calve_exercises=list(calves[2])

#Hashmap #1 creation
exercise_dict={}
exercise_dict["Back"]=back_exercises
exercise_dict["Abs"]=abdominal_exercises
exercise_dict["Chest"]=chest_exercises
exercise_dict["Biceps"]=bicep_exercises
exercise_dict["Quadriceps"]=quad_exercises
exercise_dict["Hamstrings"]=hams_exercises
exercise_dict["Triceps"]=tricep_exercises
exercise_dict["Shoulders"]=shoulder_exercises
exercise_dict["Lower Back"]=lower_back_exercises
exercise_dict["Calves"]=calve_exercises
exercise_dict

#Hashmap #2 creation

routine_dict={}
routine_dict["Upper"]=["Chest", "Back", "Biceps", "Triceps", "Shoulders", "Abs"]
routine_dict["Lower"]=["Lower Back", "Calves","Hamstrings", "Quadriceps", "Abs"]
routine_dict["Legs"]=["Lower Back", "Calves","Hamstrings", "Quadriceps", "Abs"]
routine_dict["FullBody"]=["Chest", "Back", "Biceps", "Triceps", "Shoulders", "Abs", "Lower Back", "Calves","Hamstrings", "Quadriceps"]
routine_dict["ChestBack"]=["Chest", "Back",  "Abs"]
routine_dict["Push"]=["Chest", "Shoulders",  "Triceps", "Abs"]
routine_dict["Pull"]=["Back", "Lower Back", "Biceps", "Abs"]
routine_dict["ShoulderArms"]=["Shoulders", "Biceps", "Triceps", "Abs"]

#bucketize the two(interlink)

my_msgone=input("What are you hitting today Upper, Lower, Pull, Push, Legs, FullBody, ChestBack, ShoulderArms? :")

workout_for_today={}
for workout in routine_dict:
    if my_msgone.lower()==workout.lower():
        if workout=="ChestBack":
            for muscle_group in routine_dict[workout]:
                if muscle_group=="Chest" or muscle_group=="Back":
                    #this is done in order to add more exercises per important muscle group
                    workout_for_today[muscle_group]=random.sample(exercise_dict[muscle_group], 2)
                else:
                    workout_for_today[muscle_group]=random.choice(exercise_dict[muscle_group])
        elif workout=="FullBody":
            routine_dict[workout]=random.sample(routine_dict[workout], 5)
            for muscle_group in routine_dict[workout]:
                workout_for_today[muscle_group]=random.choice(exercise_dict[muscle_group])
        elif workout=="Pull":
            for muscle_group in routine_dict[workout]:
                if muscle_group=="Back":
                    workout_for_today[muscle_group]=random.sample(exercise_dict[muscle_group], 2)
                else:
                    workout_for_today[muscle_group]=random.choice(exercise_dict[muscle_group])
                
        elif workout=="ShoulderArms":
            for muscle_group in routine_dict[workout]:
                if muscle_group=="Shoulders":
                    workout_for_today[muscle_group]=random.sample(exercise_dict[muscle_group], 2)
                else:
                    workout_for_today[muscle_group]=random.choice(exercise_dict[muscle_group])
                
        elif workout=="Upper":
            routine_dict[workout]=random.sample(routine_dict[workout], 5) 
            for muscle_group in routine_dict[workout]:
                workout_for_today[muscle_group]=random.choice(exercise_dict[muscle_group])
        elif workout=="Push":
            for muscle_group in routine_dict[workout]:
                if muscle_group=="Chest":
                    workout_for_today[muscle_group]=random.sample(exercise_dict[muscle_group], 2)
                else:
                    workout_for_today[muscle_group]=random.choice(exercise_dict[muscle_group])
        else:
            for muscle_group in routine_dict[workout]:
                workout_for_today[muscle_group]=random.choice(exercise_dict[muscle_group])
            
workout_for_today
lux=copy.copy(workout_for_today)  
saying=[ "This workout ain't nothing but a peanut:", "Hit this in the jim now:"]
print(random.choice(saying), lux, "Here are your rep ranges:", "Sets X Reps:", random.randint(3,5), "X", random.randint(8,12))



# messageone=client.messages.create(to=my_cell, from_=my_twilio, body=my_msgone)




What are you hitting today Upper, Lower, Pull, Push, Legs, FullBody, ChestBack, ShoulderArms? :legs
Hit this in the jim now: {'Lower Back': 'Quadruped Alternating Superman', 'Calves': 'Single-Leg Calf Press', 'Hamstrings': 'Hip Lift', 'Quadriceps': 'Barbell Side Lunge', 'Abs': 'Cross Crunch w/ Medicine Ball'} Here are your rep ranges: Sets X Reps: 3 X 12


In [4]:
#FiGURE OUT IMPORT OF GOOGLE API CLIENT, SO YOU COULD GO THROUGH The folder directory 

In [9]:
#PART TWO, INPUT NECESSARY
from Google import Create_Service

sa=gspread.service_account(filename="service_account.json")


client_list=["Prototype"]
for client in client_list:
    sh=sa.open(client)
wks=sh.worksheet("Schedule")
scraped_list=wks.get("A2:B8")

curr_date = date.today()
workout_day=calendar.day_name[curr_date.weekday()]

my_msgtwo=input("Are you working out today(Yes or No)? :")


try:
    daily_schedule={day[0]:day[1] for day in scraped_list}
    #for every spreadsheet in folder directory, 
    for day in daily_schedule:
        if day==workout_day:
            if daily_schedule[day].lower()==my_msgtwo.lower():
                print("Carpe Diem!")
            elif my_msgtwo.lower()=="no" and daily_schedule[day].lower()=="yes":
                print("Stop lying bruh, excuses and excuses")
            elif my_msgtwo.lower()=="yes" and daily_schedule[day].lower()=="no":
                print("Take a rest day bud")
except:
    print("fill out the spreadsheet")
       


# messagetwo=client.messages.create(to=my_cell, from_=my_twilio, body=my_msgtwo)

Are you working out today(Yes or No)? :yes
Take a rest day bud
